<a href="https://colab.research.google.com/github/Nancy-Shi/Complex_Networks/blob/main/Behaviour_Model_Two_States.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Behavior Model with Two States

There are two states in this layer of the model. NP represents the state of no protection, while P represents the state of with protection. The rate of transition from state P to NP, p, depends on the information layer. The rate from NP to P is 1-p. The transition rate of a node is also affected by the degree of the hyperedge size and number of active spreader/stiflers. The larger the hyperedge sizes, the more rapid the rate. The bigger number of active neighbors, the faster the rate.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import random
import math as math
from math import log
import seaborn as sns
import pandas as pd


## Part 1: Hypergraph Generation
The following steps generate a hyper graph using the XGI/HyperNetX python package,  following power-law degree distribution for predifined number of nodes n, number of hyperedges num_hyper_edges, degree exponent gamma, using a configuration model with data stored in a dictionary.

In [2]:
# Step 1: Generate Degree Sequence
def generate_degree_sequence(n, gamma, kmin):
    # Generate a random set from the power law distribution
    u = np.random.uniform(size=n)
    degrees = np.ceil((1.0 - u) ** (-1.0 / (gamma - 1.0)))

    # Adjust degrees based on the minimum and maximum degree values
    kmax = int(np.sqrt(n))
    # kmax = int(1.5*n**(1/4)) # max degree allowed is 1.5*n^(1/4)
    degrees = degrees[(degrees >= kmin) & (degrees <= kmax)].astype(int)

    # Truncate or pad the sequence to match the length specified
    if len(degrees) >= n:
        degrees = degrees[:n]
    else:
        degrees = np.concatenate((degrees, np.full(n - len(degrees), kmin)))

    return degrees.tolist()

# Step 2: Generate Hyper Edge Size Sequence
def generate_hyper_edge_sizes(degrees, num_hyper_edges):
    total_degrees = sum(degrees)
    hyper_edge_sizes = []

    # Calculate the average size for each hyper edge
    avg_size = total_degrees // num_hyper_edges
    remainder = total_degrees % num_hyper_edges

    # Define the range for the random distribution
    min_size = 1  # Lower bound of the range
    max_size = len(degrees)-num_hyper_edges  # Upper bound of the range

    # Generate hyper edge sizes
    for _ in range(num_hyper_edges):
        size = random.randint(min_size, max_size)
        hyper_edge_sizes.append(size)

    return hyper_edge_sizes


# Step 3: Create Copies of Nodes
def create_node_copies(degrees):
    node_copies = []
    for i, degree in enumerate(degrees):
        for _ in range(degree):
            node_copies.append(i)
    return node_copies

# Step 4: Create Copies of Hyper Edges
def create_hyper_edge_copies(hyper_edge_sizes):
    hyper_edge_copies = []
    for i, size in enumerate(hyper_edge_sizes):
        for _ in range(size):
            hyper_edge_copies.append(i)
    return hyper_edge_copies

# Step 5: Randomly Pair Copies
def randomly_pair_copies(node_copies, hyper_edge_copies):
    random.shuffle(node_copies)
    random.shuffle(hyper_edge_copies)
    pairs = []
    for i in range(len(node_copies)):
        pairs.append((node_copies[i], hyper_edge_copies[i]))
    return pairs

# Step 6: Convert Bipartite Graph to A Hypergraph Dictionary
def convert_to_hypergraph(pairs):
    hypergraph = {}
    for pair in pairs:
        node, hyper_edge = pair
        if hyper_edge in hypergraph:
            hypergraph[hyper_edge].append(node)
        else:
            hypergraph[hyper_edge] = [node]
    return hypergraph


In [3]:
def build_hypergraph(n, gamma, kmin, num_hyper_edges):
    # Step 1: Generate Degree Sequence
    degrees = generate_degree_sequence(n, gamma, kmin)
    print("Degree Sequence: ", degrees)

    # Step 2: Generate Hyper Edge Size Sequence
    hyper_edge_sizes = generate_hyper_edge_sizes(degrees, num_hyper_edges)
    print("Hyper Edge Sizes: ", hyper_edge_sizes)

    # Step 3: Create Copies of Nodes
    node_copies = create_node_copies(degrees)

    # Step 4: Create Copies of Hyper Edges
    hyper_edge_copies = create_hyper_edge_copies(hyper_edge_sizes)

    # Step 5: Randomly Pair Copies
    pairs = randomly_pair_copies(node_copies, hyper_edge_copies)

    # Step 6: Convert Bipartite Graph to Hypergraph
    hyperedge_dict = convert_to_hypergraph(pairs)

    # Print the resulting hypergraph
    print("Hypergraph Dictionary: ", hyperedge_dict)

    return degrees, hyperedge_dict

In [9]:
#!pip install hypernetx
import hypernetx as hnx

In [6]:
# Test 2
n2 =400  # Number of nodes
gamma2 = 2.5  # Power-law exponent
kmin2 = 3  # Minimum degree
num_hyper_edges2 = 100  # Desired number of hyper edges

degrees2, hyperedge_dict2 = build_hypergraph(n2, gamma2, kmin2, num_hyper_edges2)
H2 = hnx.Hypergraph(hyperedge_dict2)

Degree Sequence:  [3, 5, 3, 3, 3, 4, 3, 4, 3, 3, 3, 3, 6, 11, 3, 3, 4, 4, 3, 3, 3, 4, 4, 3, 7, 3, 3, 3, 5, 4, 3, 3, 18, 3, 5, 4, 9, 3, 3, 5, 3, 8, 3, 3, 3, 16, 6, 6, 10, 4, 3, 4, 3, 6, 15, 3, 5, 3, 5, 3, 3, 4, 3, 5, 7, 4, 6, 7, 10, 6, 3, 3, 7, 3, 5, 4, 6, 4, 6, 5, 13, 4, 4, 13, 12, 15, 3, 4, 6, 4, 3, 3, 3, 6, 3, 3, 3, 11, 5, 4, 5, 4, 6, 3, 6, 7, 10, 7, 3, 5, 4, 3, 7, 3, 7, 3, 3, 3, 3, 3, 3, 4, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 

## Part 2: Assign Behavior Status
NP represents the state of no protection, while P represents the state of with protection.

In [15]:
def random_protection_assignment(hypergraph, fraction_protected):
    # Get the list of nodes from the hypergraph
    nodes = list(hypergraph.nodes())

    # Calculate the number of nodes to protect
    num_nodes_to_protect = int(len(nodes) * fraction_protected)

    # Randomly choose nodes to protect
    nodes_to_protect = random.sample(nodes, num_nodes_to_protect)

    # Initialize the protection status dictionary
    protection_status = {}

    # Assign protection status to each node
    for node in nodes:
        if node in nodes_to_protect:
            protection_status[node] = "P"  # Protected node
        else:
            protection_status[node] = "NP"  # Non-protected node

    return protection_status

In [16]:
# Test:
fraction_protected = 0.3  # 30% of nodes will be protected
protection_status_dict = random_protection_assignment(H2, fraction_protected)
print(protection_status_dict)

{0: 'P', 1: 'NP', 2: 'NP', 3: 'NP', 4: 'NP', 5: 'NP', 6: 'P', 7: 'P', 8: 'P', 9: 'P', 10: 'P', 11: 'NP', 12: 'P', 13: 'P', 14: 'NP', 15: 'P', 16: 'NP', 17: 'P', 18: 'NP', 19: 'NP', 20: 'NP', 21: 'NP', 22: 'NP', 23: 'NP', 24: 'NP', 25: 'NP', 26: 'NP', 27: 'NP', 28: 'P', 29: 'NP', 30: 'P', 31: 'NP', 32: 'NP', 33: 'NP', 34: 'NP', 35: 'NP', 36: 'NP', 37: 'NP', 38: 'NP', 39: 'P', 40: 'NP', 41: 'NP', 42: 'P', 43: 'NP', 44: 'P', 45: 'NP', 46: 'NP', 47: 'NP', 48: 'P', 49: 'NP', 50: 'NP', 51: 'NP', 52: 'NP', 53: 'P', 54: 'NP', 55: 'NP', 56: 'P', 57: 'NP', 58: 'P', 59: 'NP', 60: 'NP', 61: 'NP', 62: 'P', 63: 'P', 64: 'NP', 65: 'NP', 66: 'NP', 67: 'NP', 68: 'NP', 69: 'NP', 70: 'NP', 71: 'NP', 72: 'P', 73: 'P', 74: 'P', 75: 'NP', 76: 'NP', 77: 'NP', 78: 'P', 79: 'NP', 80: 'NP', 81: 'NP', 82: 'P', 83: 'NP', 84: 'P', 85: 'NP', 86: 'P', 87: 'P', 88: 'NP', 89: 'NP', 90: 'NP', 91: 'NP', 92: 'NP', 93: 'P', 94: 'P', 95: 'P', 96: 'P', 97: 'P', 98: 'NP', 99: 'NP', 100: 'NP', 101: 'P', 102: 'NP', 103: 'P', 1


## Part 3: Assign Threshold
The following steps assigns a threshold value to each node in the network. The threshold follows a uniform or normal distribution with predefined mean (mu) and standard deviation (sigma).

In [17]:
# Defines the parameters to be used
mu = 0.1
sigma = 0.05

# Function to assign thresholds to the individual nodes
def assign_thresholds(hypergraph):
    NV = hypergraph.order()
    Ltre = {}

    for node in hypergraph.nodes():
          # Uniform distribution: #
          #Ltre[node] = np.random.uniform()
          # Normal distrution
          while True:
              threshold = random.gauss(mu, sigma)
              if 0 < threshold < 1:
                  break
          Ltre[node] = threshold

    return Ltre

In [18]:
Ltre2 = assign_thresholds(H2)

print("Threshold List for Nodes: ", Ltre2 )

Threshold List for Nodes:  {0: 0.13304221820782536, 1: 0.11589766856362269, 2: 0.0851055335566057, 3: 0.005690524298323937, 4: 0.07684011527489212, 5: 0.10980004005063698, 6: 0.07200209167915078, 7: 0.0742809139941035, 8: 0.11832285675520623, 9: 0.1531200845482554, 10: 0.21224597711365786, 11: 0.1325021654580802, 12: 0.10335669417135813, 13: 0.13445049983305105, 14: 0.17029234966036155, 15: 0.04288523605269208, 16: 0.09188745130664945, 17: 0.026764537104196948, 18: 0.08292618720115462, 19: 0.07176898598180188, 20: 0.07642791312275923, 21: 0.05432454481919219, 22: 0.08275138875971086, 23: 0.09314202737473697, 24: 0.07433807880983806, 25: 0.06733640665798846, 26: 0.14179023724093825, 27: 0.17263892961002325, 28: 0.028169848022859073, 29: 0.05558130904247996, 30: 0.15330854954097567, 31: 0.1288670796997261, 32: 0.16083050207755029, 33: 0.09786466252517526, 34: 0.08999700004353564, 35: 0.14475711674148267, 36: 0.09085140142369501, 37: 0.1552898765701739, 38: 0.05735376646722336, 39: 0.0933

## Part 4: Behavior Change
The rate of transition from state P to NP, p, depends on the information layer. The rate from NP to P is 1-p. The transition rate of a node is also affected by the degree of the hyperedge size and number of active spreader/stiflers. The larger the hyperedge sizes, the more rapid the rate. The bigger number of active neighbors, the faster the rate.